In [25]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [26]:
file_path = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/raw_prices_ubereats_ca_ff_03222024.csv"
ca_ff = pd.read_csv(file_path)
file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/ca_zip_county.csv"
ca_zip_count = pd.read_csv(file_path_2)

In [27]:
#Drop all the columns we don't need 
ca_ff_ = ca_ff.drop(columns=['Unnamed: 0', 'inputted_location', 'number_of_ratings', 'restaurant_distance'])

Exploring missing values 

In [28]:
#Nan values 
nan_count = ca_ff.isnull().sum()
print(nan_count)

Unnamed: 0               0
restaurant_name          0
menu_item                0
menu_item_price          0
restaurant_location    250
inputted_location        0
restaurant_rating        0
number_of_ratings        0
restaurant_distance    235
dtype: int64


In [29]:
#There are two inputted locations that have no restaurant address: 

#Mcdonald's at 1950 Parkside Dr, Concord, CA 94519 

# Wendy's at 1600 Truxtun Ave, Bakersfield, CA 93306

#Let's drop those restaurants

ca_ff_ = ca_ff_.dropna(subset=['restaurant_location'])

In [30]:
#restaurant_rating cleaning 

rows_with_mi = ca_ff_['restaurant_rating'].str.contains('mi').sum()

print("Number of rows with 'mi' in restaurant rating:", rows_with_mi)

#Remove restaurants with no valid rating 
ca_ff_['restaurant_rating'] = ca_ff_['restaurant_rating'].str.replace(r'.*mi$', '0', regex=True)

Number of rows with 'mi' in restaurant rating: 3742


In [31]:
#converting data types 
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].astype('string')
ca_ff_['menu_item'] = ca_ff_['menu_item'].str.replace(r'\s+', ' ', regex=True)
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].astype('string')
ca_ff_['restaurant_rating'] = ca_ff_['restaurant_rating'].str.strip().astype(float)

In [32]:
#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_

,restaurant_name,menu_item,menu_item_price,restaurant_location,restaurant_rating
0,McDonald,medium french fries,4.69,"1330 jackson st, oakland, ca, 94612, us",4.6
1,McDonald,10 pc chicken mcnuggets,6.69,"1330 jackson st, oakland, ca, 94612, us",4.6
2,McDonald,big mac,6.79,"1330 jackson st, oakland, ca, 94612, us",4.6
3,McDonald,2 cheeseburger meal,10.99,"1330 jackson st, oakland, ca, 94612, us",4.6
4,McDonald,cheeseburger,3.49,"1330 jackson st, oakland, ca, 94612, us",4.6
...,...,...,...,...,...
148708,The Habit,ranch,0.63,"1855 e daily dr, camarillo, ca, 93010, us",4.8
148709,The Habit,bbq sauce,0.63,"1855 e daily dr, camarillo, ca, 93010, us",4.8
148710,The Habit,teriyaki sauce,0.63,"1855 e daily dr, camarillo, ca, 93010, us",4.8
148711,The Habit,spicy red pepper sauce,0.63,"1855 e daily dr, camarillo, ca, 93010, us",4.8


In [33]:
def price_list(x):
    return list(x)

In [34]:
def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

McDonald's 

In [35]:
#Filter to just McDonalds
ca_ff_mcd = ca_ff_[ca_ff_['restaurant_name'] == 'McDonald']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_mcd = ca_ff_mcd.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_mcd.columns = [' '.join(col).strip() for col in grouped_mcd.columns.values]


#Second part of grouping 
mcd_lst = ['big mac', 'big mac meal', 'cheeseburger', 'hamburger', 'medium french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_mcd = ca_ff_mcd[ca_ff_mcd['menu_item'].isin(mcd_lst)].sort_values('menu_item')
menu_items_mcd = menu_items_mcd.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_mcd_2 = menu_items_mcd.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_mcd_2[['specialty_item', 'combo', 'cheeseburger', 'hamburger', 'fries']] = grouped_mcd_2['menu_item_price'].apply(pd.Series)
grouped_mcd_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_mcd = pd.merge(grouped_mcd, grouped_mcd_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_mcd

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,specialty_item,combo,cheeseburger,hamburger,fries
0,McDonald,"100 e redlands blvd, san bernardino, ca, 92408...",6.522047,5.59,4.706970,4.6,144,7.89,13.99,4.39,3.49,5.89
1,McDonald,"1000 e 4th st, long beach, ca, 90802, us",7.439231,5.19,5.679119,4.7,141,8.29,14.39,4.39,3.89,5.29
2,McDonald,"1000 real rd, bakersfield, ca, 93309, us",6.029685,3.69,6.196706,4.9,147,6.19,11.39,3.09,2.79,4.59
3,McDonald,"10051 bruceville road, elk grove, ca, 95757, us",7.812595,5.19,9.957629,4.4,148,7.69,11.79,4.09,3.69,4.99
4,McDonald,"1009 n wilson way, stockton, ca, 95205-4217, us",7.812595,5.19,9.957629,4.5,149,7.69,11.79,4.09,3.69,4.99
...,...,...,...,...,...,...,...,...,...,...,...,...
223,McDonald,"981 lakeville hwy, petaluma, ca, 94952, us",5.817907,4.29,3.995190,4.7,294,8.69,14.79,4.19,3.79,5.69
224,McDonald,"990 alamo dr, vacaville, ca, 95687, us",8.243897,5.29,9.697837,4.7,156,8.49,12.99,3.59,3.49,5.29
225,McDonald,"chester &amp; brundage, bakersfield, ca, 93304...",5.382640,4.29,3.762974,4.5,139,7.49,11.89,3.99,3.09,5.19
226,McDonald,"mission blvd, santa rosa, ca, 95409, us",5.952109,4.24,4.262860,4.6,154,7.99,12.69,3.89,3.39,4.89


Jack in the Box 

In [36]:
#Filter to just Jack in the Box
ca_ff_jack = ca_ff_[ca_ff_['restaurant_name'] == 'Jack in the Box']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_jack = ca_ff_jack.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_jack.columns = [' '.join(col).strip() for col in grouped_jack.columns.values]

# #Second part of grouping 
jack_lst = ['jr jumbo jack', 'jr jumbo jack cheeseburger', 'jumbo jack', 'large french fry', 'large jumbo jack combo']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_jack = ca_ff_jack[ca_ff_jack['menu_item'].isin(jack_lst)].sort_values('menu_item')
menu_items_jack = menu_items_jack.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_jack_2 = menu_items_jack.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_jack_2[['hamburger', 'cheeseburger', 'specialty_item', 'fries', 'combo']] = grouped_jack_2['menu_item_price'].apply(pd.Series)
grouped_jack_2.drop(columns=['menu_item_price'], inplace=True)

# #Merging the grouped dfs together 
merged_jack = pd.merge(grouped_jack, grouped_jack_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_jack

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,hamburger,cheeseburger,specialty_item,fries,combo
0,Jack in the Box,"10 union ave, bakersfield, ca, 93307, us",8.563382,7.49,4.284847,4.4,136,3.74,4.11,7.49,5.36,13.73
1,Jack in the Box,"100 s. state college blvd., anaheim, ca, 92806...",7.567080,6.49,3.870589,4.6,137,3.24,3.49,6.11,4.74,11.98
2,Jack in the Box,"1003 east california avenue, bakersfield, ca, ...",10.281912,8.55,5.750189,4.3,136,4.74,4.99,9.11,5.99,18.35
3,Jack in the Box,"1004 steele ln, santa rosa, ca, 95403, us",7.798346,6.61,4.124576,4.1,133,3.61,3.99,6.24,4.86,12.60
4,Jack in the Box,"10110 sierra ave, fontana, ca, 92335, us",6.957868,5.99,3.610198,4.5,136,3.11,3.47,5.39,4.31,10.90
...,...,...,...,...,...,...,...,...,...,...,...,...
200,Jack in the Box,"9179 elk grove florin road, elk grove, ca, 956...",8.526483,8.11,4.316240,4.3,145,3.74,4.11,6.99,4.99,13.85
201,Jack in the Box,"960 n ventura rd, oxnard, ca, 93030, us",7.543577,6.25,3.857673,4.7,137,3.49,3.74,6.86,4.49,12.23
202,Jack in the Box,"9620 e stockton blvd, elk grove, ca, 95624, us",8.278085,7.11,4.247735,4.6,141,3.36,3.61,6.61,4.99,13.48
203,Jack in the Box,"986 woodside rd, redwood city, ca, 94061, us",7.623066,6.29,4.120327,5.0,137,2.27,2.53,5.99,4.74,12.35


Wendy's

In [37]:
#Filter to just Wendy's
ca_ff_wendy = ca_ff_[ca_ff_['restaurant_name'] == 'Wendy']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_wendy = ca_ff_wendy.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_wendy.columns = [' '.join(col).strip() for col in grouped_wendy.columns.values]

# #Second part of grouping 
wendy_lst = ['daves combo', 'daves single', 'french fries', 'jr cheeseburger', 'jr hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_wendy = ca_ff_wendy[ca_ff_wendy['menu_item'].isin(wendy_lst)].sort_values('menu_item')
menu_items_wendy = menu_items_wendy.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_wendy_2 = menu_items_wendy.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_wendy_2[['combo', 'specialty_item', 'fries', 'cheeseburger', 'hamburger']] = grouped_wendy_2['menu_item_price'].apply(pd.Series)
grouped_wendy_2.drop(columns=['menu_item_price'], inplace=True)

merged_wendy = pd.merge(grouped_wendy, grouped_wendy_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_wendy


,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,combo,specialty_item,fries,cheeseburger,hamburger
0,Wendy,"1001 redwood, vallejo, ca, 94590, us",5.160893,4.225,2.768295,4.5,376,0.0,6.29,0.0,2.39,2.43
1,Wendy,"10020 arlington avenue, riverside, ca, 92503, us",6.968704,5.780,3.223025,4.3,95,0.0,8.16,0.0,3.65,3.40
2,Wendy,"10040 chapman ave., garden grove, ca, 92840, us",6.696182,5.480,3.283198,4.5,190,0.0,8.41,0.0,3.65,3.65
3,Wendy,"1012 n. state college blvd., anaheim, ca, 9280...",6.675818,5.480,3.280588,4.7,95,0.0,8.16,0.0,3.65,3.65
4,Wendy,"102 encinites blvd, encinitas, ca, 92024, us",6.926545,6.090,3.130902,4.8,95,0.0,8.28,0.0,3.89,3.40
...,...,...,...,...,...,...,...,...,...,...,...,...
130,Wendy,"9180 rosedale hwy, bakersfield, ca, 93312, us",7.088000,6.580,3.484735,4.2,95,0.0,8.77,0.0,3.40,2.92
131,Wendy,"924 pleasant grove blvd, roseville, ca, 95678, us",6.971875,5.360,4.213493,4.4,194,0.0,7.35,0.0,3.25,3.60
132,Wendy,"960 eastlake parkway, chula vista, ca, 91914, us",6.755636,5.970,3.103376,4.7,95,0.0,8.04,0.0,3.65,3.28
133,Wendy,"960 w. el norte, escondido, ca, 92026, us",5.537273,4.890,2.543573,4.9,95,0.0,6.59,0.0,2.99,2.69


Burger King

In [38]:
#Filter to just Burger King 
ca_ff_bk = ca_ff_[ca_ff_['restaurant_name'] == 'Burger King']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_bk = ca_ff_bk.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bk.columns = [' '.join(col).strip() for col in grouped_bk.columns.values]

# #Second part of grouping 
#they don't have a plain hamburger FOR NOW using whopper jr 
bk_lst = ['cheeseburger', 'french fries', 'whopper', 'whopper jr', 'whopper meal']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bk = ca_ff_bk[ca_ff_bk['menu_item'].isin(bk_lst)].sort_values('menu_item')
menu_items_bk = menu_items_bk.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bk_2 = menu_items_bk.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bk_2[['cheeseburger', 'fries', 'specialty_item', 'hamburger', 'combo']] = grouped_bk_2['menu_item_price'].apply(pd.Series)
grouped_bk_2.drop(columns=['menu_item_price'], inplace=True)

merged_bk = pd.merge(grouped_bk, grouped_bk_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bk

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,cheeseburger,fries,specialty_item,hamburger,combo
0,Burger King,"10055 cedar avenue, bloomington, ca, 92316, us",8.970256,6.69,5.813355,4.6,156,2.79,4.89,10.29,5.39,15.29
1,Burger King,"1009 n ben maddox way, visalia, ca, 93292, us",9.749524,7.74,6.230228,4.8,84,3.49,5.29,9.89,3.79,16.69
2,Burger King,"1030 mclaughlin ave., san jose, ca, 95122, us",8.656750,7.34,6.048298,4.3,80,2.63,4.31,9.23,4.67,15.58
3,Burger King,"1042 north carpenter road, modesto, ca, 95351, us",9.540633,7.79,5.848853,3.9,79,3.89,5.59,9.79,5.39,16.39
4,Burger King,"111, colma, ca, 94014, us",8.927654,6.69,6.021679,4.4,162,2.89,5.69,9.89,4.89,16.69
...,...,...,...,...,...,...,...,...,...,...,...,...
162,Burger King,"950 west a street, hayward, ca, 94541, us",8.522716,6.79,5.814128,4.7,81,2.73,4.61,9.86,4.86,14.98
163,Burger King,"969 east francisco blvd., san rafael, ca, 9490...",8.897927,6.69,5.990826,4.4,164,2.89,5.69,9.89,4.89,16.69
164,Burger King,"97 bonita road, chula vista, ca, 91910, us",9.098250,6.99,5.875383,4.5,80,2.79,4.89,10.29,5.39,15.29
165,Burger King,"972 el camino real, south san francisco, ca, 9...",9.273889,8.04,6.123416,4.8,144,4.09,4.19,8.29,5.29,15.29


Shake Shack

In [39]:
#Filter to just Shake Shack
ca_ff_shake = ca_ff_[ca_ff_['restaurant_name'] == 'Shake Shack']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_shake = ca_ff_shake.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_shake.columns = [' '.join(col).strip() for col in grouped_shake.columns.values]


#Second part of grouping 
shake_lst = ['cheeseburger', 'fries', 'hamburger', 'shackburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_shake = ca_ff_shake[ca_ff_shake['menu_item'].isin(shake_lst)].sort_values('menu_item')
menu_items_shake = menu_items_shake.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_shake_2 = menu_items_shake.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_shake_2[['cheeseburger', 'fries', 'hamburger', 'specialty_item']] = grouped_shake_2['menu_item_price'].apply(pd.Series)
grouped_shake_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_shake = pd.merge(grouped_shake, grouped_shake_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_shake['combo'] = np.nan

Sonic

In [40]:
#FLAG many missing prices for fries and 
#Filter to just Sonic 
ca_ff_sonic = ca_ff_[ca_ff_['restaurant_name'] == 'Sonic']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_sonic = ca_ff_sonic.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_sonic.columns = [' '.join(col).strip() for col in grouped_sonic.columns.values]


#Second part of grouping 
sonic_lst = ['fries', 'quarter pound double cheeseburger', 'supersonic double cheeseburger', 'supersonic double cheeseburger combo']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_sonic = ca_ff_sonic[ca_ff_sonic['menu_item'].isin(sonic_lst)].sort_values('menu_item')
menu_items_sonic = menu_items_sonic.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_sonic_2 = menu_items_sonic.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_sonic_2[['fries', 'cheeseburger', 'specialty_item', 'combo']] = grouped_sonic_2['menu_item_price'].apply(pd.Series)
grouped_sonic_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_sonic = pd.merge(grouped_sonic, grouped_sonic_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_sonic['hamburger'] = np.nan
merged_sonic

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,fries,cheeseburger,specialty_item,combo,hamburger
0,Sonic,"101 main st, vista, ca, 92084, us",5.787187,5.420,2.593374,4.2,164,0.0,4.87,10.36,0.0,NaN
1,Sonic,"10354 trinity parkway, stockton, ca, 95219, us",4.717353,3.950,2.132040,4.9,84,0.0,4.87,9.02,0.0,NaN
2,Sonic,"10515 mission gorge rd, santee, ca, 92071, us",5.787188,5.420,2.593374,3.9,164,0.0,4.87,10.36,0.0,NaN
3,Sonic,"1060 west renaissance parkway, rialto, ca, 923...",5.266944,4.810,2.310804,4.4,344,0.0,4.75,8.89,0.0,NaN
4,Sonic,"1070 e huntington dr, duarte, ca, 91010, us",5.233333,4.870,2.251423,4.6,86,0.0,4.87,8.53,0.0,NaN
5,Sonic,"11370 4th street, rancho cucamonga, ca, 91730, us",4.403889,3.650,2.127671,4.4,258,0.0,4.75,8.28,0.0,NaN
6,Sonic,"11780 amargosa road, victorville, ca, 92392, us",4.344118,3.890,2.094945,4.8,160,0.0,NaN,NaN,NaN,NaN
7,Sonic,"1202 magnolia ave, corona, ca, 92881, us",5.424688,5.115,2.382008,4.7,246,0.0,4.50,9.75,0.0,NaN
8,Sonic,"1215 grand ave, san marcos, ca, 92078, us",5.402188,5.115,2.373903,4.3,82,0.0,4.50,9.75,0.0,NaN
9,Sonic,"1400 north park boulevard, pittsburg, ca, 9456...",5.898611,4.870,2.428605,4.3,86,0.0,4.87,10.24,0.0,NaN


Five Guys 

In [41]:
#Filter for Five Guys
ca_ff_five = ca_ff_[ca_ff_['restaurant_name'] == 'Five Guys']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_five = ca_ff_five.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_five.columns = [' '.join(col).strip() for col in grouped_five.columns.values]


#Second part of grouping 
five_lst = ['cheeseburger', 'little cheeseburger', 'little hamburger', 'regular fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_five = ca_ff_five[ca_ff_five['menu_item'].isin(five_lst)].sort_values('menu_item')
menu_items_five = menu_items_five.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_five_2 = menu_items_five.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_five_2[['specialty_item', 'cheeseburger', 'hamburger', 'fries']] = grouped_five_2['menu_item_price'].apply(pd.Series)
grouped_five_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_five = pd.merge(grouped_five, grouped_five_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_five['combo'] = np.nan
merged_five

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,specialty_item,cheeseburger,hamburger,fries,combo
0,Five Guys,"100 crescent drive, pleasant hill, ca, 94523, us",8.319655,8.63,3.386572,4.7,145,13.55,10.79,9.59,6.95,NaN
1,Five Guys,"10060 alabama street, redlands, ca, 92374, us",8.327931,8.27,3.146880,4.8,145,13.19,10.43,9.23,7.19,NaN
2,Five Guys,"10140 carmenita rd., whittier, ca, 90605, us",9.116552,10.07,3.492919,4.7,29,14.15,11.27,10.07,7.91,NaN
3,Five Guys,"10540 sierra avenue, fontana, ca, 92337, us",8.458571,8.69,3.104703,4.7,112,13.19,10.43,9.23,7.19,NaN
4,Five Guys,"1101 pacific avenue, santa cruz, ca, 95060, us",8.567778,8.87,2.904410,4.4,27,12.83,10.43,9.59,7.31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Five Guys,"915 playa ave, sand city, ca, 93955, us",8.420000,8.87,2.950847,4.7,28,12.83,10.43,9.59,7.31,NaN
96,Five Guys,"9257 laguna springs dr, elk grove, ca, 95758, us",9.908621,10.07,3.707268,4.7,58,15.47,12.47,11.15,8.75,NaN
97,Five Guys,"929 n. milliken avenue suite 103, ontario, ca,...",8.484286,8.75,3.089914,4.6,56,13.19,10.43,9.23,7.19,NaN
98,Five Guys,"933 e. hillsdale, foster city, ca, 94404, us",8.737143,9.11,3.314437,4.6,112,13.79,10.31,9.83,7.19,NaN


The Habit

In [42]:
ca_ff_habit = ca_ff_[ca_ff_['restaurant_name'] == 'The Habit']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_habit = ca_ff_habit.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_habit.columns = [' '.join(col).strip() for col in grouped_habit.columns.values]


#Second part of grouping 
habit_lst = ['2 original double char meal', 'charburger', 'charburger with cheese', 'double char', 'french fries']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_habit = ca_ff_habit[ca_ff_habit['menu_item'].isin(habit_lst)].sort_values('menu_item')
menu_items_habit = menu_items_habit.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_habit_2 = menu_items_habit.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_habit_2[['combo', 'hamburger', 'cheeseburger', 'specialty_item','fries']] = grouped_habit_2['menu_item_price'].apply(pd.Series)
grouped_habit_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_habit = pd.merge(grouped_habit, grouped_habit_2, on=['restaurant_name', 'restaurant_location'], how='inner')
#merged_five['combo'] = np.nan
merged_habit

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,combo,hamburger,cheeseburger,specialty_item,fries
0,The Habit,"101 e orangethorpe ave, fullerton, ca, 92832, us",10.800886,8.740,9.751003,4.7,79,17.24,6.99,8.11,9.24,4.36
1,The Habit,"10370 sepulveda blvd, mission hills, usa, los ...",10.736329,8.740,9.752436,4.6,79,16.86,6.99,8.11,8.86,4.36
2,The Habit,"1040 grant road suite 370, mountain view, ca, ...",10.800886,8.740,9.751003,4.8,79,17.24,6.99,8.11,9.24,4.36
3,The Habit,"1078 e brokaw rd, san jose, ca, 95131, us",10.788354,8.740,9.753327,4.8,158,17.24,6.99,8.11,9.24,4.36
4,The Habit,"1080 monroe st, albany, ca, 94706, us",11.037436,8.990,9.801813,4.7,390,17.49,7.24,8.36,9.49,4.36
...,...,...,...,...,...,...,...,...,...,...,...,...
133,The Habit,"933 pleasant grove blvd, roseville, ca, 95678, us",10.413291,7.810,9.688853,4.8,158,16.24,6.74,7.86,8.74,4.11
134,The Habit,"969 e prosperity ave, tulare, ca, 93274, us",10.912152,8.990,9.767491,4.9,79,17.24,7.24,8.36,9.24,4.36
135,The Habit,"9750 sierra avenue, fontana, ca, 92335, us",11.030897,8.990,9.804341,4.6,234,17.49,7.24,8.36,9.49,4.36
136,The Habit,"985 betteravia road, santa maria, ca, 93454, us",9.673281,7.495,10.173021,4.4,64,16.24,6.74,7.86,8.74,4.11


Carl's Jr.

In [43]:
#Filter to Carl's Jr 

ca_ff_carls = ca_ff_[ca_ff_['restaurant_name'] == 'Carls Jr']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count'
}

grouped_carls = ca_ff_carls.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_carls.columns = [' '.join(col).strip() for col in grouped_carls.columns.values]


#Second part of grouping 
carls_lst = ['big hamburger', 'california classic double cheeseburger', 'naturalcut french fries', 'single big carl', 'single big carl combo']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_carls = ca_ff_carls[ca_ff_carls['menu_item'].isin(carls_lst)].sort_values('menu_item')
menu_items_carls = menu_items_carls.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_carls_2 = menu_items_carls.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_carls_2[['hamburger', 'cheeseburger', 'fries', 'specialty_item','combo']] = grouped_carls_2['menu_item_price'].apply(pd.Series)
grouped_carls_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_carls = pd.merge(grouped_carls, grouped_carls_2, on=['restaurant_name', 'restaurant_location'], how='inner')
#merged_five['combo'] = np.nan
merged_carls

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,hamburger,cheeseburger,fries,specialty_item,combo
0,Carls Jr,"1 hallidie plaza, san francisco, ca, 94102, us",9.172692,7.240,4.770822,4.5,255,4.51,7.02,5.70,NaN,NaN
1,Carls Jr,"1000 farmers ln, santa rosa, ca, 95405, us",9.931622,10.070,4.593499,4.2,79,4.05,5.15,4.66,NaN,NaN
2,Carls Jr,"101 ferrari ranch rd, lincoln, ca, 95648, us",10.951848,10.755,5.163306,4.8,98,6.82,6.18,4.50,8.37,14.18
3,Carls Jr,"101 s union ave, bakersfield, ca, 93307, us",11.165000,10.755,5.341912,4.5,98,6.82,6.18,4.50,8.37,14.18
4,Carls Jr,"10770 macarthur blvd, oakland, ca, 94605, us",7.464340,5.920,4.151253,4.8,170,3.69,5.75,4.65,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
93,Carls Jr,"871 marina village pkwy, alameda, ca, 94501, us",7.576081,6.340,4.172734,4.8,535,3.94,6.16,4.99,NaN,NaN
94,Carls Jr,"8919 limonite ave, riverside, ca, 92509, us",6.759184,5.550,3.205968,5.0,87,5.15,5.15,4.55,NaN,NaN
95,Carls Jr,"899 cherry ave, san bruno, ca, 94066, us",7.412453,5.920,4.053991,4.7,340,3.69,5.75,4.65,NaN,NaN
96,Carls Jr,"915 main st, oakley, ca, 94561, us",10.401154,10.570,5.248624,4.7,84,6.82,6.18,4.50,NaN,NaN


In [44]:
#Stack all restaurants
uber_eats_ff_rnd1_prices = pd.concat([merged_mcd, merged_jack, merged_wendy, merged_shake, merged_bk, merged_sonic, merged_carls, merged_habit, merged_five])

Add location columns 

In [45]:
pattern = r",\s*([a-zA-Z]{2}),\s*(\d{5}(?:-\d{4})?),"

def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Apply the function to extract state and zip code
uber_eats_ff_rnd1_prices[['state', 'zip']] = uber_eats_ff_rnd1_prices['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))
uber_eats_ff_rnd1_prices['zip'] = uber_eats_ff_rnd1_prices['zip'].str.split('-').str[0].astype(int)

In [46]:
ca_zip_count = ca_zip_count[['zip', 'county']]
uber_eats_ff_rnd1_prices = uber_eats_ff_rnd1_prices.merge(ca_zip_count, on = 'zip')

Dataset info

In [47]:
specific_date = datetime.strptime('03222024', '%m%d%Y')
# Assign the datetime object to the entire 'date' column
uber_eats_ff_rnd1_prices['date'] = specific_date
uber_eats_ff_rnd1_prices['uber_eats'] = 1 
uber_eats_ff_rnd1_prices['post_policy'] = 0 
uber_eats_ff_rnd1_prices['fast_food'] = 1 
uber_eats_ff_rnd1_prices['local'] = 0 

In [48]:
uber_eats_ff_rnd1_prices

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,specialty_item,combo,cheeseburger,hamburger,fries,state,zip,county,date,uber_eats,post_policy,fast_food,local
0,McDonald,"100 e redlands blvd, san bernardino, ca, 92408...",6.522047,5.59,4.706970,4.6,144,7.89,13.99,4.39,3.49,5.89,ca,92408,San Bernardino,2024-03-22,1,0,1,0
1,Burger King,"205 east redlands, san bernardino, ca, 92408, us",8.970256,6.69,5.813355,4.1,78,10.29,15.29,2.79,5.39,4.89,ca,92408,San Bernardino,2024-03-22,1,0,1,0
2,McDonald,"1000 e 4th st, long beach, ca, 90802, us",7.439231,5.19,5.679119,4.7,141,8.29,14.39,4.39,3.89,5.29,ca,90802,Los Angeles,2024-03-22,1,0,1,0
3,McDonald,"640 long beach blvd, long beach, ca, 90802, us",5.264737,3.79,4.269700,4.4,134,6.46,12.09,3.04,2.95,4.59,ca,90802,Los Angeles,2024-03-22,1,0,1,0
4,Jack in the Box,"652 atlantic ave, long beach, ca, 90802, us",8.162628,7.49,4.660305,4.3,137,6.86,13.74,3.74,3.11,3.74,ca,90802,Los Angeles,2024-03-22,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,Five Guys,"5650 balboa ave suite 101, san diego, ca, 9211...",9.900345,9.95,3.705692,4.6,58,15.47,NaN,12.47,11.15,8.75,ca,92117,San Diego,2024-03-22,1,0,1,0
1138,Five Guys,"6285 pats ranch rd., mira loma, ca, 91752, us",8.290690,8.27,3.177414,4.5,58,13.19,NaN,10.43,9.23,7.19,ca,91752,Riverside,2024-03-22,1,0,1,0
1139,Five Guys,"6660 lone tree way, brentwood, ca, 94513, us",8.319655,8.63,3.386572,4.8,29,13.55,NaN,10.79,9.59,6.95,ca,94513,Contra Costa,2024-03-22,1,0,1,0
1140,Five Guys,"8420 balboa blvd. suite 115, northridge, ca, 9...",9.875517,10.07,3.699241,4.8,29,15.47,NaN,12.47,11.15,8.75,ca,91343,Los Angeles,2024-03-22,1,0,1,0


In [49]:
#Save as csv 
uber_eats_ff_rnd1_prices.to_csv('processed_prices_ubereats_ca_ff_03222024.csv', index = True)